Reference - https://www.youtube.com/watch?v=TKjjlp5_r7o

In [1]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import pandas as pd
import os
import numpy as np

In [2]:
parent_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
folder_path = r'Data\PreprocessedData'
net_path = os.path.join(parent_dir, folder_path)

In [3]:
df_train_english = pd.read_csv(os.path.join(net_path, 'english_train_preprocess.csv'))
df_dev_english = pd.read_csv(os.path.join(net_path, 'english_dev_preprocess.csv'))
df_test_english = pd.read_csv(os.path.join(net_path, 'english_test_preprocess.csv'))

In [4]:
train_english_sentences = df_train_english['preprocessed_text'].tolist()
dev_english_sentences = df_dev_english['preprocessed_text'].tolist()
test_english_sentences = df_test_english['preprocessed_text'].tolist()

In [5]:
english_sentences = train_english_sentences + dev_english_sentences + test_english_sentences

In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [7]:
lemmatized_texts = lemmatization(english_sentences)

In [8]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['tiktok', 'radiate', 'gay', 'chaotic', 'energy', 'love']


In [9]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]
chaotic


In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

## Visualize

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=3)
vis

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.129096  0.295697       1        1  39.123068
0      0.185373 -0.263191       2        1  31.921424
1     -0.314469 -0.032506       3        1  28.955508, topic_info=       Term         Freq        Total Category  logprob  loglift
13   matter  4865.000000  4865.000000  Default   3.0000   3.0000
12     life  4591.000000  4591.000000  Default   2.0000   2.0000
26   people  3756.000000  3756.000000  Default   1.0000   1.0000
32       so  2136.892179  2137.641156   Topic1  -3.6675   0.9381
150      go  1688.442567  1689.216161   Topic1  -3.9031   0.9380
136   think  1503.876374  1504.652040   Topic1  -4.0188   0.9379
41     just  2204.271105  2211.162194   Topic1  -3.6365   0.9353
13   matter  4864.914495  4865.612791   Topic2  -2.6414   1.1417
12     life  4590.405360  4591.106972   Topic2  -2.6995   1.1417
16    black  3553.691855  3554.398771   Topic2  -2.9555   1.1417
26   people  3755.219793  3756.008774   Topic2  -2.9003   1.1417
3      love  1387.801246  1388.563098   Topic3  -3.7982   1.2389
286     see  1055.212529  1056.024553   Topic3  -4.0722   1.2386
46     know   979.255749   980.017242   Topic3  -4.1469   1.2386
8       get  1330.224991  1698.336764   Topic3  -3.8406   0.9951, token_table=      Topic      Freq    Term
term                         
16        2  0.999888   black
8         2  0.216683     get
8         3  0.783119     get
150       1  0.999280      go
41        1  0.996761    just
41        2  0.003166    just
46        3  0.998962    know
12        2  0.999759    life
3         3  0.999594    love
13        2  0.999874  matter
26        2  0.999731  people
286       3  0.999030     see
32        1  0.999700      so
136       1  0.999567   think, R=3, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])